In [52]:
from bs4 import BeautifulSoup
import requests
import time
import re
import glob
import os
import codecs
import mysql.connector
from pandas import DataFrame
import pandas as pd


In [53]:
headers = {'User-Agent': 'Mozilla/5.0'}
folder = os.getcwd() + '\\reviewmeta_pages\\'
if not os.path.exists(folder):
    os.makedirs(folder)

# read the file we just stored
source_product = pd.read_csv('source_product_test.csv', sep=',')

problematic = []

In [51]:
source_product.head()

,Source,product
0,amazon.ca,B00EAPCCW0
1,amazon.ca,B000RIA95G
2,amazon.uk,B0057H1HZI
3,amazon.ca,B00CWR7HFK
4,amazon.ca,B00QTCUV0C


## Functions 

Given a link this script will fetch data from reviewmeta

In [29]:
def checksource(url):
    if (".fr" in url):
        return "amazon-fr"
    if (".ca" in url):
        return "amazon-ca"
    if (".de" in url):
        return "amazon-de"
    if (".uk" in url):
        return "amazon-uk"
    else:
        return "amazon"

### webscraping from reviewmeta

In [5]:
def data_per_product(cat,number_of_rev,soup,product):
    output = {}
    trust, rev_rate, rev_title, rev_det, rev_link_rm,rev_txt_am,rev_lnk_am = ([] for i in range(7))
    
    if(cat == "good"):
        trust_soup = soup.find_all("div",class_  = "col-sm-2 col-xs-12 text-center review-pass-badge")
        if(len(trust_soup)<number_of_rev):
            temp = soup.find_all("div",class_ = "col-sm-2 col-xs-12 text-center review-warn-badge")
            trust_soup = trust_soup + temp

    elif(cat == "bad"):
        trust_soup = soup.find_all("div",class_  = "col-sm-2 col-xs-12 text-center review-fail-badge")
        if(len(trust_soup)<number_of_rev):
            temp = soup.find_all("div",class_ = "col-sm-2 col-xs-12 text-center review-warn-badge")
            trust_soup = trust_soup + temp

    
    for i in range(0,number_of_rev):
        
        trust.append      (int(re.findall(r'[0-9]+',str(trust_soup[i].text))[0])/100)                  #convert trust to 0 to 1 scale
        rev_rate.append   (re.findall(r'[0-5]',str(soup.find_all("div",class_  = "col-md-1 col-xs-2 bw-rating")[i].text))[0])
        rev_title.append  (soup.find_all("div",class_  = "col-md-11 col-xs-10 show-title")[i].text)
        rev_det.append    (soup.find_all("div",class_  = "col-sm-10 col-xs-12")[i].text)
        rev_link_rm.append(soup.find_all("label",class_= "written-by")[i].find("a").get('href'))
        rev_txt_am.append (soup.find_all('div',class_  = "show-actual-review") [i].text)
        rev_lnk_am.append (soup.find_all('div',class_  = "show-actual-review") [i].find("a").get('href'))

    output["product"]          = product
    output["trust"]            = trust   
    output["review_rating"]    = rev_rate
    output["review_title"]     = rev_title
    output["reviewer_details"] = rev_det
    output["reviewer_link_RM"] = rev_link_rm
    output["rvwr_text_Amazon"] = rev_txt_am
    output["rvwr_link_Amazon"] = rev_lnk_am 
    return(output)

### reviwer profile detials clean up

In [6]:
def rev_details_dict(review_temp):
    rev_dict = {"Verified_Purchases": 0, "Nvr_verified_reviewer"   : 0, "Contains_rep_phrases": 0,
                  "high_vol_day_rev": 0, "Critical_Rev_rating"     : 0, "Take_backs"          : 0, 
                 "Take_backs_rating": 0, "Overrep_part"            : 0, "Overrep_wrd_cnt"     : 0,
                 "Easy_grade_rating": 0, "Overlapping_rev_history" : 0, "Brand_Rep_freq"      : 0,
                 "Brand_rep_rating" : 0, "One_hit"                 : 0 }

    for rev_metric in review_temp:

        if("Unverified Purchaser" in rev_metric):
            rev_dict["Verified_Purchases"] = 0

        if("Verified Purchaser" in rev_metric):
            rev_dict["Verified_Purchases"] = 1

    #Ex: Never-Verified Reviewer
        if("Never-Verified Reviewer" in rev_metric):
            rev_dict["Nvr_verified_reviewer"] = 1

        if("Contains repetitive phrases" in rev_metric):
            rev_dict["Contains_rep_phrases"] = 1

        if("Created on a high volume day" in rev_metric):
            rev_dict["high_vol_day_rev"] = 1

        if("Critical Reviewer" in rev_metric):
            rev_dict["Critical_Rev_rating"] = float(rev_metric[-4:-1])

        if("Take-Back Reviewer" in rev_metric):
            rev_dict["Take_backs"]         = int(re.findall(r'[0-9]+',rev_metric)[0])
            rev_dict["Take_backs_rating"]  = float(rev_metric[-4:-1])

    #EX: Overrepresented Participation (posted 2 reviews)        
        if("Overrepresented Participation" in rev_metric):
            rev_dict["Overrep_part"]       = int(re.findall(r'[0-9]+',rev_metric)[0])   

    #EX: Overrepresented word count (3 words)       
        if("Overrepresented word count" in rev_metric):
            rev_dict["Overrep_wrd_cnt"]    = int(re.findall(r'[0-9]+',rev_metric)[0])   

    #Ex: Easy Grader (avg. rating: 4.8)        
        if("Easy Grader" in rev_metric):
            rev_dict["Easy_grade_rating"] = float(rev_metric[-4:-1])

    #Ex: Overlapping Review History: reviewed 1 of 9 top products        
        if("Overlapping Review History" in rev_metric):
            rev_dict["Overlapping_rev_history"] = int(re.findall(r'[0-9]+',rev_metric)[0])/int(re.findall(r'[0-9]+',rev_metric)[1])

    #Brand Repeater (2 of 53 reviews for this brand; avg rating: 5.0)
        if("Brand Repeater" in rev_metric):
            rev_dict["Brand_Rep_freq"]     = (re.findall(r'[0-9]+',rev_metric)[0])+"/"+(re.findall(r'[0-9]+',rev_metric)[1])
            rev_dict["Brand_rep_rating"]   = float(rev_metric[-4:-1])

        if("One-Hit Wonder" in rev_metric):
             rev_dict["One_hit"] = 1 
    return rev_dict


### give review dataframe for a single prduct 

In [7]:
def get_review_df_per_soup(soup, productname):
    soup_good     = soup.find("div", {"id": "good-reviews"})
    no_of_good    = len(soup_good.find_all("div",id = "sample_reviews"))  
    soup_bad      = soup.find("div", {"id": "bad-reviews"})
    no_of_bad     = len(soup_bad.find_all("div",id = "sample_reviews"))

    good_reviews_sum = pd.DataFrame.from_dict(data_per_product("good",no_of_good,soup_good,productname))
    bad_reviews_sum  = pd.DataFrame.from_dict(data_per_product("bad",no_of_bad,soup_bad,productname))

    review_total              = good_reviews_sum.append(bad_reviews_sum).reset_index(drop=True)
    review_total["Amazon_ID"] = review_total["rvwr_link_Amazon"].apply(lambda x: x[(x.rfind("/"))+1:]) 
    
   # print(review_total)
    
    # split individual review deets into attributes
    all_review_dict=[]
    for i in range(no_of_good + no_of_bad):
        review_temp = review_total["reviewer_details"][i].split("\n")
        rev_details = rev_details_dict(review_temp)
        all_review_dict.append(rev_details)

    #merge all the dictonaries with same key
    review_attributes = {}
    for k in all_review_dict[0]:
        review_attributes[k] = [d[k] for d in all_review_dict]

    review_total= review_total.join(pd.DataFrame.from_dict(review_attributes))
    return review_total

## Downloading review meta pages with time lag on local

In [34]:
for i in range(0,len(source_product)):
    time.sleep(1) #Hold 1 seconds before the next scrape.
    num=str(i)
  #  url = source_product["URL"][i]
    product = source_product["product"][i]
    source = checksource (source_product["Source"][i])
    path = os.getcwd() + '\\reviewmeta_pages\\'
    filename = product +"_"+ source + '.htm'
    review_meta_url = 'https://reviewmeta.com/'+ source +'/'+ product
    print(review_meta_url)
    response = requests.get(review_meta_url, headers = headers)
    time.sleep(2)
    with open(path + filename, 'w', encoding="utf8") as file:
        file.write(response.text)

print('Completed')

https://reviewmeta.com/amazon-ca/B00EAPCCW0
https://reviewmeta.com/amazon-ca/B000RIA95G
https://reviewmeta.com/amazon-uk/B0057H1HZI
https://reviewmeta.com/amazon-ca/B00CWR7HFK
https://reviewmeta.com/amazon-ca/B00QTCUV0C
https://reviewmeta.com/amazon-uk/B01ATS8NUQ
https://reviewmeta.com/amazon-ca/B00063496C
https://reviewmeta.com/amazon-fr/B005BOMTVS
https://reviewmeta.com/amazon-uk/B000RXVJEQ
https://reviewmeta.com/amazon-uk/B005BOMT72
Completed


In [47]:
review_dataframe_mega = pd.DataFrame()
files = os.listdir(path)
for file_name in files:
    print(file_name)
    f = open("{}{}".format(path,file_name),"r", encoding="utf-8").read()  #creating the path 
    soup = BeautifulSoup(f)
    if (soup.find_all("div",id = "sample_reviews") != []):
        review_dataframe_mega = pd.concat([review_dataframe_mega, get_review_df_per_soup(soup, file_name.split("_")[-2])], ignore_index=True)
    else:
        problematic.append(file_name.split("_")[0])

B00063496C_amazon-ca.htm
B000RIA95G_amazon-ca.htm
B000RXVJEQ_amazon-uk.htm
B0057H1HZI_amazon-uk.htm
B005BOMT72_amazon-uk.htm
B005BOMTVS_amazon-fr.htm
B00CWR7HFK_amazon-ca.htm
B00EAPCCW0_amazon-ca.htm
B00QTCUV0C_amazon-ca.htm
B01ATS8NUQ_amazon-uk.htm


In [48]:
problematic

['B005BOMT72', 'B00QTCUV0C']

In [50]:
review_dataframe_mega.to_csv("review_dataframe_mega.csv")

In [104]:
review_dataframe_mega.describe()

,trust,Verified_Purchases,Nvr_verified_reviewer,Contains_rep_phrases,high_vol_day_rev,Critical_Rev_rating,Take_backs,Take_backs_rating,Overrep_part,Overrep_wrd_cnt,Easy_grade_rating,Overlapping_rev_history,Brand_Rep_freq,Brand_rep_rating,One_hit
count,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.0,372.0,372.000000
mean,0.557312,0.975806,0.013441,0.252688,0.177419,1.181720,0.580645,0.962634,1.728495,2.588710,3.106452,0.017025,0.0,0.0,0.255376
std,0.465495,0.153857,0.115308,0.435139,0.382538,1.686506,2.012482,1.879490,4.014656,11.838545,2.341864,0.072571,0.0,0.0,0.436660
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
50%,0.790000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.700000,0.000000,0.0,0.0,0.000000
75%,1.000000,1.000000,0.000000,1.000000,0.000000,3.125000,0.000000,0.000000,1.000000,0.000000,5.000000,0.000000,0.0,0.0,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,4.300000,26.000000,5.000000,32.000000,90.000000,5.000000,0.666667,0.0,0.0,1.000000


#for whatever reason the reveiw id is not unique lenght wise : pls see if this can be done nicer

Apologies for the multiple ifs, pls see if these can be done more efficiently 

### Approach 

1) Make this a general function that takes product url as inout and give the review dataframe as output 


2) scrape reviewer_link_RM and get that data